# Tutorial of fMRI state visualisation using workbench

Author: Yiming Wei

Date: 22nd January 2024

The aim of this tutorial is to provide a first example of fMRI data visualisation using osl-dynamics toolbox within OHBA analysis group. We will utilise the HCP workbench and tools in osl_dynamics.analysis. This tutorial should enable us (Chet, Ben, Brian and myself) to compare dynamic FC models (HMM, Dynemo and mDynemo) trained on different large-scale datasets (HCP, UKBiobank and Cam-CAN). Several prerequisites are listed below:

1. BMRC cluster. 

We need to set up workbench properly on BMRC cluster so that `wb_command` can be run properly in the command line. The easiest way is to run `module load ConnectomeWorkbench` before starting this tutorial. (TODO: It seems to me that this command does not work properly if run in jupyter notebook or python subprocess. Why?).
P.S. There's a `setup` function in `osl_dynamics.analysis.workbench.py` to setup workbench, which requires to specify the workbench path. However, the BMRC CPUs work on two architectures (refer to this [page](https://www.medsci.ox.ac.uk/for-staff/resources/bmrc/python-on-the-bmrc-cluster) for more info), which means that the path is dependent on the actual node. (Maybe different ideas?).

2. Brain surface map.

By default, osl-dynamics uses ParcellationPilot. HCP S1200 map, which is available on the HCP database, might be another option.

3. Spatial map.
Obtained from group-level spatial ICA. Should be specific to the dataset.
HCP: surface map.
UKB: was volumetric, but now surface map - I'll email Fidel.
CAM-CAN: not sure.

4. State mean activation and FC map.
These are the parameters in the observation model. For example, if we train HMM model with K states on ICA results with N channels, the state mean activation should be a numpy array with shape K * N, the FC map should be a 3D numpy array with shape K * N * N.

We give a simple example below. Note some of the functions are in `rotation/utils.py`, which is my own function toolbox.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
from rotation.utils import IC2surface,first_eigenvector
from osl_dynamics.analysis.workbench import render

In [4]:
# Specify the directories
spatial_map_dir = '../../../data/spatial_maps/groupICA_3T_HCP1200_MSMAll_d25.ica/melodic_IC.dscalar.nii'
mean_activation_dir = '../../../results_202310/HMM_ICA_25_state_8/state_means.npy'
fc_dir = '../../../results_202310/HMM_ICA_25_state_8/state_correlations.npy'
save_dir ='./'

In [12]:
# Load files
spatial_surface_map = nib.load(spatial_map_dir)
state_means = np.load(mean_activation_dir)
correlations = np.load(fc_dir)


In [13]:
# Calculate surface map of mean activation
mean_activation_surface_map = IC2surface(spatial_surface_map,state_means.T)
mean_activation_surface_map.to_filename(f'{save_dir}mean_activation_surface_map.dscalar.nii')

In [14]:
# Rank-one approximation
r1_approxs = []
for i in range(len(correlations)):
    correlation = correlations[i,:,:]
    r1_approxs.append(first_eigenvector(correlation))
r1_approxs = np.array(r1_approxs)
np.save(f'{save_dir}r1_approx_FC.npy', r1_approxs)

# Calculate surface map of FC
FC_degree_surface_map = IC2surface(spatial_surface_map,r1_approxs.T)
FC_degree_surface_map.to_filename(f'{save_dir}FC_surface_map.dscalar.nii')

In [17]:
# Render
render(nii=f'{save_dir}mean_activation_surface_map.dscalar.nii',
       save_dir=f'{save_dir}/visualisation',
       gui=False,
       inflation=0,
       image_name='./visualisation/mean_mid_thickness',
       input_cifti=True
       )
render(nii=f'{save_dir}FC_surface_map.dscalar.nii',
       save_dir=f'{save_dir}/visualisation',
       gui=False,
       inflation=0,
       image_name='./visualisation/FC_mid_thickness',
       input_cifti=True
       )

Saving images: 100%|██████████████████████████████| 8/8 [00:07<00:00,  1.02it/s]
